In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt


In [ ]:
# ------------------------------
# Load data
# ------------------------------
penguins = sns.load_dataset("penguins").dropna()
penguins["is_adelie"] = (penguins["species"] == "Adelie").astype(int)

X = penguins[["bill_length_mm", "bill_depth_mm",
              "flipper_length_mm", "body_mass_g"]].values
y = penguins["is_adelie"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# ------------------------------
# Logistic Regression
# ------------------------------
lr = LogisticRegression(penalty=None)
lr.fit(X_train_scaled, y_train)
prob_lr = lr.predict_proba(X_test_scaled)[:,1]

In [ ]:
# ------------------------------
# Neural Network = 1-layer LR
# ------------------------------
tf.random.set_seed(0)

nn = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

nn.compile(optimizer=tf.keras.optimizers.Adam(0.01),
           loss='binary_crossentropy',
           metrics=['accuracy'])

nn.fit(X_train_scaled, y_train, epochs=10000, verbose=0)
prob_nn = nn.predict(X_test_scaled).flatten()

In [ ]:
# ------------------------------
# Compare predictions
# ------------------------------
plt.figure(figsize=(6,6))
plt.scatter(prob_lr, prob_nn, alpha=0.7)
plt.plot([0,1], [0,1], 'r--')
plt.xlabel("Logistic Regression probability")
plt.ylabel("Neural Net probability")
plt.title("Neural Net vs Logistic Regression predicted probabilities")
plt.grid(True)
plt.show()

In [ ]:
# ------------------------------
# Print coefficients side by side
# ------------------------------
print("Logistic Regression coefficients:")
print(lr.coef_)
print("Intercept:", lr.intercept_)

print("\nNeural Network weights:")
w, b = nn.layers[0].get_weights()
print(w.reshape(-1))
print("Bias:", b)